# Exporting data from BigQuery to Google cloud storage

In this notebook, we export BigQuery data to GCS so that we can reuse our Keras model that was developed on CSV data.

Uncomment the following line if you are running the notebook locally:

In [1]:
# !sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
%load_ext google.cloud.bigquery

In [3]:
import os

from google.cloud import bigquery

In [4]:
# Change with your own bucket and project below:
BUCKET =  "buddie_rec_data"
PROJECT = "buddie-270710"

OUTDIR = "gs://{bucket}/buddieRec/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

## cretae BigQuery tables

In [5]:
bq = bigquery.Client(project = PROJECT)
dataset = bigquery.Dataset(bq.dataset("buddieRec"))

try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset already exists


In [6]:
%%bigquery

CREATE OR REPLACE TABLE buddieRec.ga_training_data AS

SELECT 
    clientId, 
    timeOnPage,
    (
    SELECT STRING_AGG(part, '/' ORDER BY index) 
    FROM UNNEST(SPLIT(pagePath, '/')) part WITH OFFSET index 
    WHERE index BETWEEN 1 AND 3
  ) contentId, 
    organization
FROM (select split(clientId,'.')[offset (0)] as clientId, 
        split(pagePath,'?')[offset(0)] as pagePath,
        timeOnPage,
        split(hostname,'.')[offset(0)] as organization
From `buddie-270710.ga_data.rec_data`)
WHERE ABS(MOD(FARM_FINGERPRINT(`clientId`),10)) < 8 AND RAND() < 1

""


In [7]:
%%bigquery

CREATE OR REPLACE TABLE buddieRec.ga_valid_data AS

SELECT 
    clientId, 
    timeOnPage,
    (
    SELECT STRING_AGG(part, '/' ORDER BY index) 
    FROM UNNEST(SPLIT(pagePath, '/')) part WITH OFFSET index 
    WHERE index BETWEEN 1 AND 3
  ) contentId,
    organization
FROM (select split(clientId,'.')[offset (0)] as clientId, 
        split(pagePath,'?')[offset(0)] as pagePath,
        timeOnPage,
        split(hostname,'.')[offset(0)] as organization
From `buddie-270710.ga_data.rec_data`)
WHERE ABS(MOD(FARM_FINGERPRINT(`clientId`),10)) = 8 AND RAND() < 1


""


In [8]:
%%bigquery

CREATE OR REPLACE TABLE buddieRec.uniqClientIds AS

SELECT 
    distinct(SPLIT(clientId, '.')[OFFSET(1)]) as clientId
FROM `buddie-270710.ga_data.rec_data`
GROUP BY clientId


""


In [9]:
%%bigquery 

CREATE OR REPLACE TABLE buddieRec.uniqOrganizations AS

SELECT 
    DISTINCT(SPLIT(hostname, '.')[OFFSET(0)])  organization 
FROM `buddie-270710.ga_data.rec_data`
GROUP BY SPLIT(hostname, '.')[OFFSET(0)] 

""


In [10]:
%%bigquery

CREATE OR REPLACE TABLE buddieRec.uniqContentIds AS

select 
    DISTINCT(SPLIT(pagePath, "?")[ORDINAL(1)]) as contentId
FROM `buddie-270710.ga_data.rec_data`
GROUP BY pagePath

""


## export the tables as CSV files

In [11]:
%%bash

echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

echo "Extracting training data to $OUTDIR"
bq --location=europe-west4 extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   buddieRec.ga_training_data \
   $OUTDIR/rec-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=europe-west4 extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   buddieRec.ga_valid_data \
   $OUTDIR/rec-valid-*.csv

echo "Extracting unique clientIds to $OUTDIR"
bq --location=europe-west4 extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   buddieRec.uniqClientIds \
   $OUTDIR/rec-clientIds.csv


echo "Extracting unique organizations to $OUTDIR"
bq --location=europe-west4 extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   buddieRec.uniqOrganizations \
   $OUTDIR/rec-orgIds.csv


echo "Extracting contentIds to $OUTDIR"
bq --location=europe-west4 extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   buddieRec.uniqContentIds \
   $OUTDIR/rec-contentIds.csv

gsutil ls -l $OUTDIR

Deleting current contents of gs://buddie_rec_data/buddieRec/data
Extracting training data to gs://buddie_rec_data/buddieRec/data
Extracting validation data to gs://buddie_rec_data/buddieRec/data
Extracting unique clientIds to gs://buddie_rec_data/buddieRec/data
Extracting unique organizations to gs://buddie_rec_data/buddieRec/data
Extracting contentIds to gs://buddie_rec_data/buddieRec/data
      5236  2021-02-06T20:02:10Z  gs://buddie_rec_data/buddieRec/data/rec-clientIds.csv
      1993  2021-02-06T20:02:19Z  gs://buddie_rec_data/buddieRec/data/rec-contentIds.csv
        73  2021-02-06T20:02:14Z  gs://buddie_rec_data/buddieRec/data/rec-orgIds.csv
     44707  2021-02-06T20:02:01Z  gs://buddie_rec_data/buddieRec/data/rec-train-000000000000.csv
      5210  2021-02-06T20:02:06Z  gs://buddie_rec_data/buddieRec/data/rec-valid-000000000000.csv
TOTAL: 5 objects, 57219 bytes (55.88 KiB)


Waiting on bqjob_r1bf39489ca5126a8_0000017778efd2cb_1 ... (2s) Current status: DONE   
Waiting on bqjob_r497016a145e6c3cd_0000017778efe438_1 ... (2s) Current status: DONE   
Waiting on bqjob_r6d8bb4d19a58d2b8_0000017778eff537_1 ... (2s) Current status: DONE   
Waiting on bqjob_r12f173ca89149776_0000017778f00672_1 ... (2s) Current status: DONE   
Waiting on bqjob_r17834b0e0f93921d_0000017778f017dc_1 ... (2s) Current status: DONE   


In [12]:
!gsutil cat gs://$BUCKET/buddieRec/data/rec-train-000000000000.csv | head -2

CommandException: No URLs matched: gs://buddie_rec_data/buddieRec/data/rec-train-000000000000.csv
